In [2]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
input_size = 3
batch_size = 5
eps = 1e-1

class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum
        self.eval_mode = False
        self.ema_mean = 0
        self.ema_var = 1

    def __call__(self, input_tensor):
        if not self.eval_mode:
            batch_mean = torch.mean(input_tensor, dim=0)
            batch_var = torch.var(input_tensor, dim=0, unbiased=False)
            batch_var_unbiased = torch.var(input_tensor, dim=0, unbiased=True)
            self.ema_mean = (1-self.momentum)*batch_mean + self.momentum*self.ema_mean
            self.ema_var = (1-self.momentum)*batch_var_unbiased + self.momentum*self.ema_var

        else:
            batch_mean = self.ema_mean
            batch_var = self.ema_var

        normed_tensor = ((input_tensor - batch_mean) / (batch_var + self.eps)**(1/2)) * self.weight + self.bias

        return normed_tensor

    def eval(self):
        self.eval_mode = True

batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

# Code validation.
all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape

print(all_correct)

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape
print(all_correct)

True
True


In [ ]:
# Custom BatchNorm layer for 4D input.

eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)


def custom_batch_norm2d(input_tensor, eps):
    normed_tensor = torch.zeros_like(input_tensor)
    for channel in range(input_channels):
        batch_mean = torch.mean(input_tensor[:, channel])
        batch_var = torch.var(input_tensor[:, channel], unbiased=False)
        normed_tensor[:, channel] = ((input_tensor[:, channel] - batch_mean) / (batch_var + eps)**(1/2))
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = batch_norm_2d(input_tensor)
custom_output = custom_batch_norm2d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)


True


In [3]:
# Custom layer (channel) normalization.

import torch
import torch.nn as nn


eps = 1e-10


def custom_layer_norm(input_tensor, eps):
    normed_tensor = torch.zeros_like(input_tensor)
    for item_number in range(input_tensor.shape[0]):
        layer_mean = torch.mean(input_tensor[item_number])
        layer_var = torch.var(input_tensor[item_number], unbiased=False)
        normed_tensor[item_number] = ((input_tensor[item_number] - layer_mean) / (layer_var + eps)**(1/2))

    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True


In [11]:
# Custom instance normalization.

import torch
import torch.nn as nn

eps = 1e-3

batch_size = 5
input_channels = 2
input_length = 30

instance_norm = nn.InstanceNorm1d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, input_length, dtype=torch.float)


def custom_instance_norm1d(input_tensor, eps):
    normed_tensor = torch.zeros_like(input_tensor)
    instance_mean = torch.mean(input_tensor, dim=2, keepdim=True)
    instance_var = torch.var(input_tensor, unbiased=False, dim=2, keepdim=True)
    normed_tensor = ((input_tensor - instance_mean) / (instance_var + eps)**(1/2))

    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = instance_norm(input_tensor)
custom_output = custom_instance_norm1d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output, atol=1e-06) and norm_output.shape == custom_output.shape)

True


In [14]:
# Custom group normalization.

import torch
import torch.nn as nn

channel_count = 6
eps = 1e-3
batch_size = 20
input_size = 2

input_tensor = torch.randn(batch_size, channel_count, input_size)


def custom_group_norm(input_tensor, groups, eps):
    normed_tensor = torch.zeros_like(input_tensor)
    for item_number in range(input_tensor.shape[0]):
        for group_idx in range(groups):
            group_size = int(input_tensor.shape[1]/groups)
            group_mean = torch.mean(input_tensor[item_number, group_idx*group_size:group_idx*group_size+group_size])
            group_var = torch.var(input_tensor[item_number, group_idx*group_size:group_idx*group_size+group_size], unbiased=False)
            normed_tensor[item_number, group_idx*group_size:group_idx*group_size+group_size] = ((input_tensor[item_number, group_idx*group_size:group_idx*group_size+group_size] - group_mean) / (group_var + eps)**(1/2))

    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for groups in [1, 2, 3, 6]:
    group_norm = nn.GroupNorm(groups, channel_count, eps=eps, affine=False)
    norm_output = group_norm(input_tensor)
    custom_output = custom_group_norm(input_tensor, groups, eps)
    all_correct &= torch.allclose(norm_output, custom_output, 1e-3)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True
